In [3]:
import tensorflow as tf
import sys
sys.path.append("../")
from federated.models.mitbih_model import create_cnn_model, create_dense_model
from federated.utils.mitbih_data_preprocessing import get_centralized_datasets


In [4]:
model = tf.keras.models.load_model("../history/logdir/test_v1")
_, test_dataset = get_centralized_datasets(data_analysis=True) 


FileNotFoundError: [Errno 2] No such file or directory: 'data/mitbih/mitbih_train.csv'

In [13]:
scores = model.evaluate((X_test),y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))



NameError: name 'X_test' is not defined

In [14]:
print(history)
fig1, ax_acc = plt.subplots()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Model - Accuracy')
plt.legend(['Training', 'Validation'], loc='lower right')
plt.show()



NameError: name 'history' is not defined

In [15]:
fig2, ax_loss = plt.subplots()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model- Loss')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()
target_names=['0','1','2','3','4']



NameError: name 'plt' is not defined

In [16]:
y_true=[]
for element in y_test:
    y_true.append(np.argmax(element))
prediction_proba=model.predict(X_test)
prediction=np.argmax(prediction_proba,axis=1)
cnf_matrix = confusion_matrix(y_true, prediction)


NameError: name 'y_test' is not defined